In [ ]:
!cat /home/ec2-user/.keras/keras.json

# Simple CNN model for MNIST

#### Imports

In [ ]:
from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential, save_mxnet_model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
import numpy as np
from keras.utils import to_channels_first

#### Parameters

In [ ]:
batch_size = 128
epochs = 5

## Data

Convert from NHW to NCHW.

In [ ]:
# load the data, and split between train and test sets
(x_train_nhw, y_train), (x_test_nhw, y_test) = mnist.load_data()
print('x_train_nhw: ', x_train_nhw.shape)
print('x_test_nhw:  ', x_test_nhw.shape)

In [ ]:
x_train_nhwc = x_train_nhw[..., np.newaxis]
x_train = to_channels_first(x_train_nhwc)
x_test_nhwc = x_test_nhw[..., np.newaxis]
x_test = to_channels_first(x_test_nhwc)
print('x_train: ', x_train.shape)
print('x_test:  ', x_test.shape)

#### Convert and rescale

In [ ]:
print("dtype: ", x_train.dtype)
print("min: ", x_train.min())
print("max: ", x_train.max())

In [ ]:
x_train = x_train.astype('float32')/255
x_test = x_test.astype('float32')/255
print("dtype: ", x_train.dtype)
print("min: ", x_train.min())
print("max: ", x_train.max())

#### Convert class vectors to binary class matrices

In [ ]:
num_classes = 10
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

## Model

In [ ]:
# find input shape without batch size
input_shape = x_train.shape[1:]

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

#### Compile and fit

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer=keras.optimizers.Adam(),
              metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))

## Evaluate

In [ ]:
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

## Save Model

In [ ]:
data_names, data_shapes = save_mxnet_model(model=model,
                                           prefix='mnist_cnn',
                                           epoch=0)